## README 
Website: https://www.eqtlgen.org/phase1.html

Paper: https://www.nature.com/articles/s41588-021-00913-zhttps://www.nature.com/articles/s41588-021-00913-z

This Jupyter notebook README covers cis-eQTL and trans-eQTL results from the eQTLGen project. The dataset includes significant files for both cis-eQTL and trans-eQTL analyses. 

Files contain various columns with information on P-value, SNP rs ID, SNP chromosome, SNP position, assessed and not assessed alleles, Z-score, ENSG and HGNC names of eQTL genes, gene chromosome, gene position, number of cohorts, number of samples, false discovery rate, and Bonferroni-corrected P-value.

The cis-eQTL analysis includes 19,250 genes expressed in blood, with SNP-gene combinations within 1Mb from the gene center and tested in at least 2 cohorts. The trans-eQTL analysis tests 19,960 genes expressed in blood and 10,317 trait-associated SNPs based on GWAS Catalog, Immunobase, and Astle et al. study. Trans-eQTL combinations have a distance of >5Mb and were tested in at least 2 cohorts.

The FDR calculation uses a pruned set of SNPs for trans-eQTL mapping and permutation-based FDR calculation. Crossmapping filters are applied to identify and remove potential artifacts in trans-eQTL results, recalculating the FDR afterward. Note that the full results file has not been filtered for cross-mapping effects, which may lead to artifacts in the data.

The code below demonstrates the process of creating a graph-based representation of the combined cis and trans-eQTL data using PyTorch Geometric. This process can be broken down into several steps:

1. Combine cis and trans dataframes: The code begins by concatenating the cis and trans dataframes into a single dataframe named 'data', which contains information on both cis-eQTL and trans-eQTL results. This combined dataset simplifies the process of working with the data and ensures that all relevant information is contained within a single data structure.

2. Create mappings for genes and SNPs: To represent the genes and SNPs as nodes in the graph, integer indices are assigned to each unique gene and SNP. This is done using dictionaries called 'gene_to_idx' and 'snp_to_idx', which map the gene and SNP identifiers to their corresponding integer indices.

3. Generate node type labels: Node type labels are created using PyTorch tensors, distinguishing between gene nodes (assigned a label of 0) and SNP nodes (assigned a label of 1). This differentiation is useful for various graph-based analyses and machine learning tasks that require knowledge of node types.

4. Create edges based on gene and SNP indices: Edges in the graph represent the relationships between genes and SNPs. These edges are created by iterating over the 'data' dataframe and extracting the corresponding gene and SNP indices from the previously created mappings. The edges are then represented as a PyTorch tensor with a long data type.

5. Convert edges to undirected: Since the relationships between genes and SNPs are undirected, the edges in the graph should also be undirected. This is achieved using the 'to_undirected()' function from PyTorch Geometric, which ensures that the graph correctly represents the underlying biology.

6. Create a PyTorch Geometric graph: Finally, the graph is created using the PyTorch Geometric 'Data' class. The node types and edge indices are used as inputs to instantiate the graph object, which can then be utilized for further analysis and visualization.

The resulting 'graph' object is a PyTorch Geometric representation of the combined cis and trans-eQTL data. The prediction task is to predict new association edges given the training edges, with the task type being link prediction. Below are a few important graph statistics:

- Number of nodes: 3681495
- Number of SNP nodes: 3664025
- Number of Gene nodes: 17470
- Number of edges: 10567450
- Number of connected components: 424
- Average degree: 5.74
- Median degree: 2.0
- Standard deviation of degree: 69.81
- Density: 0.0000015594
- Assortativity: -0.2267915607

The prediction task is to predict new association edges given the training edges, with the task type being link prediction. The data splitting is random while maintaining an equal proportion of cis- and trans- associations.

## Data Setup 

### Libraries

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score

import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import to_undirected, negative_sampling

import networkx as nx
from ogb.io import DatasetSaver
from ogb.linkproppred import LinkPropPredDataset

In [2]:
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")

PyTorch version: 2.0.0+cu118
PyTorch Geometric version: 2.3.1


### Load data

In [3]:
# Read files
cis = pd.read_csv("sig-cis.txt", sep='\t')
trans = pd.read_csv("sig-trans.txt", sep='\t')

In [4]:
# Print cis
cis.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP
0,3.271700e-310,rs12230244,12,10117369,T,A,200.7534,ENSG00000172322,CLEC12A,12,10126104,34,30596,0.0,4.166200e-302
1,3.271700e-310,rs12229020,12,10117683,G,C,200.6568,ENSG00000172322,CLEC12A,12,10126104,34,30596,0.0,4.166200e-302
2,3.271700e-310,rs61913527,12,10116198,T,C,200.2654,ENSG00000172322,CLEC12A,12,10126104,34,30598,0.0,4.166200e-302
3,3.271700e-310,rs2594103,12,10115428,T,C,200.0420,ENSG00000172322,CLEC12A,12,10126104,34,30598,0.0,4.166200e-302
4,3.271700e-310,rs12231833,12,10118428,A,G,199.9508,ENSG00000172322,CLEC12A,12,10126104,34,30592,0.0,4.166200e-302


In [5]:
# Print trans
trans.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP
0,1.128895e-308,rs3811444,1,248039451,T,C,-72.1091,ENSG00000166086,JAM3,11,133980358,37,31684,0.0,6.659500e-302
1,1.128895e-308,rs35340377,1,248038210,A,G,-65.4239,ENSG00000166086,JAM3,11,133980358,37,31684,0.0,6.659500e-302
2,1.128895e-308,rs705705,12,56435504,C,G,60.9665,ENSG00000265688,MAFG-AS1,17,79887167,9,10740,0.0,6.659500e-302
3,1.128895e-308,rs1131017,12,56435929,C,G,59.0847,ENSG00000265688,MAFG-AS1,17,79887167,20,15015,0.0,6.659500e-302
4,1.128895e-308,rs10876864,12,56401085,G,A,58.8148,ENSG00000265688,MAFG-AS1,17,79887167,19,14930,0.0,6.659500e-302


### Create graphs

In [6]:
# For the cis dataframe
cis_genes = cis['Gene'].unique()
cis_snps = cis['SNP'].unique()
cis_gene_to_idx = {gene: idx for idx, gene in enumerate(cis_genes)}
cis_snp_to_idx = {snp: idx + len(cis_genes) for idx, snp in enumerate(cis_snps)}

cis_node_types = torch.tensor([0] * len(cis_genes) + [1] * len(cis_snps), dtype=torch.long)

cis_edges = cis.apply(lambda row: (cis_gene_to_idx[row['Gene']], cis_snp_to_idx[row['SNP']]), axis=1)
cis_edges = torch.tensor(list(cis_edges), dtype=torch.long).t().contiguous()

cis_edges = to_undirected(cis_edges)

cis_graph = Data(x=cis_node_types.view(-1, 1), edge_index=cis_edges)

print(cis_graph)



# For the trans dataframe
trans_genes = trans['Gene'].unique()
trans_snps = trans['SNP'].unique()
trans_gene_to_idx = {gene: idx for idx, gene in enumerate(trans_genes)}
trans_snp_to_idx = {snp: idx + len(trans_genes) for idx, snp in enumerate(trans_snps)}

trans_node_types = torch.tensor([0] * len(trans_genes) + [1] * len(trans_snps), dtype=torch.long)

trans_edges = trans.apply(lambda row: (trans_gene_to_idx[row['Gene']], trans_snp_to_idx[row['SNP']]), axis=1)
trans_edges = torch.tensor(list(trans_edges), dtype=torch.long).t().contiguous()

trans_edges = to_undirected(trans_edges)

trans_graph = Data(x=trans_node_types.view(-1, 1), edge_index=trans_edges)

print(trans_graph)



# Combine the cis and trans dataframes
data = pd.concat([cis, trans], ignore_index=True)

# Function to filter the lowest 20% of associations
def filter_lowest_20_percent(df):
    threshold = np.percentile(df['BonferroniP'], 20)
    return df[df['BonferroniP'] <= threshold]

# Filter cis and trans dataframes
cis_filtered = filter_lowest_20_percent(cis)
trans_filtered = filter_lowest_20_percent(trans)

# Remove filtered data from original dataframes
cis_remaining = cis[~cis.index.isin(cis_filtered.index)]
trans_remaining = trans[~trans.index.isin(trans_filtered.index)]

# Test that all values in 'BonferroniP' column of cis_filtered are lower than cis_remaining
assert all(cis_filtered['BonferroniP'] <= cis_remaining['BonferroniP'].min()), "Values in cis_filtered are not all lower than cis_remaining"

# Test that all values in 'BonferroniP' column of trans_filtered are lower than trans_remaining
assert all(trans_filtered['BonferroniP'] <= trans_remaining['BonferroniP'].min()), "Values in trans_filtered are not all lower than trans_remaining"

# Create mappings for genes and SNPs to integer indices
genes = data['Gene'].unique()
snps = data['SNP'].unique()
gene_to_idx = {gene: idx for idx, gene in enumerate(genes)}
snp_to_idx = {snp: idx + len(genes) for idx, snp in enumerate(snps)}

# Create node type labels
node_types = torch.tensor([0] * len(genes) + [1] * len(snps), dtype=torch.long)

# Create edges
edges = data.apply(lambda row: (gene_to_idx[row['Gene']], snp_to_idx[row['SNP']]), axis=1)
edges = torch.tensor(list(edges), dtype=torch.long).t().contiguous()

# Convert edges to undirected
edges = to_undirected(edges)

# Create the PyTorch Geometric graph
graph = Data(x=node_types.view(-1, 1), edge_index=edges)

print(graph)

Data(x=[3680443, 1], edge_index=[2, 21015328])
Data(x=[10151, 1], edge_index=[2, 119572])
Data(x=[3681495, 1], edge_index=[2, 21134900])


## Baseline Models

### Data Splitting

In [7]:
# Use filtered data as training data
train_data = pd.concat([cis_filtered, trans_filtered], ignore_index=True)

# Split remaining cis and trans dataframes into validation and test sets
cis_val, cis_test = train_test_split(cis_remaining, test_size=0.5, random_state=42)
trans_val, trans_test = train_test_split(trans_remaining, test_size=0.5, random_state=42)

# Concatenate the respective sets
val_data = pd.concat([cis_val, trans_val], ignore_index=True)
test_data = pd.concat([cis_test, trans_test], ignore_index=True)

# Create edge indices for train, validation, and test sets
edge_index_train = train_data.apply(lambda row: (gene_to_idx[row['Gene']], snp_to_idx[row['SNP']]), axis=1)
edge_index_val = val_data.apply(lambda row: (gene_to_idx[row['Gene']], snp_to_idx[row['SNP']]), axis=1)
edge_index_test = test_data.apply(lambda row: (gene_to_idx[row['Gene']], snp_to_idx[row['SNP']]), axis=1)

edge_index_train = torch.tensor(list(edge_index_train), dtype=torch.long).t().contiguous()
edge_index_val = torch.tensor(list(edge_index_val), dtype=torch.long).t().contiguous()
edge_index_test = torch.tensor(list(edge_index_test), dtype=torch.long).t().contiguous()

graph_train = Data(x=graph.x, edge_index=edge_index_train)
graph_val = Data(x=graph.x, edge_index=edge_index_val)
graph_test = Data(x=graph.x, edge_index=edge_index_test)

print(graph_train)
print(graph_val)
print(graph_test)

Data(x=[3681495, 1], edge_index=[2, 2113514])
Data(x=[3681495, 1], edge_index=[2, 4226968])
Data(x=[3681495, 1], edge_index=[2, 4226968])


### GCN

In [ ]:
torch.cuda.empty_cache()

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(1, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# Train and evaluate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(hidden_channels=32).to(device)

graph_train = graph_train.to(device)
graph_val = graph_val.to(device)
graph_test = graph_test.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    z = model(graph_train.x.float(), graph_train.edge_index)
    pos = (z[edge_index_train[0]] * z[edge_index_train[1]]).sum(dim=-1)
    neg_edge_index = negative_sampling(graph_train.edge_index, num_nodes=graph_train.num_nodes, num_neg_samples=graph_train.edge_index.size(1))
    neg = (z[neg_edge_index[0]] * z[neg_edge_index[1]]).sum(dim=-1)

    logits = torch.cat([pos, neg], dim=0)
    targets = torch.tensor([1] * pos.size(0) + [0] * neg.size(0), dtype=torch.float32).to(device)

    loss = F.binary_cross_entropy_with_logits(logits, targets)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(edge_index):
    model.eval()
    with torch.no_grad():
        z = model(graph_test.x.float(), graph_test.edge_index)
        pos = torch.sigmoid(torch.sum(z[edge_index[0]] * z[edge_index[1]])).view(-1)
        neg_edge_index = negative_sampling(edge_index, num_nodes=graph_test.num_nodes, num_neg_samples=edge_index.size(1))
        neg = torch.sigmoid(torch.sum(z[neg_edge_index[0]] * z[neg_edge_index[1]])).view(-1)

        preds = np.concatenate([pos.cpu().numpy(), neg.cpu().numpy()])
        true_labels = np.concatenate([np.ones_like(pos.cpu().numpy()), np.zeros_like(neg.cpu().numpy())])

        roc_auc = roc_auc_score(true_labels, preds)

        return roc_auc

for epoch in range(200):
    loss = train()
    print(f"Epoch: {epoch + 1}, Loss: {loss:.4f}")

val_roc_auc = evaluate(graph_val.edge_index)
test_roc_auc = evaluate(graph_test.edge_index)

print(f"Validation ROC-AUC: {val_roc_auc:.10f}")
print(f"Test ROC-AUC: {test_roc_auc:.10f}")

Epoch: 1, Loss: 0.8496
